In [8]:
# Import necessary utilities and functions
from dslmodel.utils.dspy_tools import init_instant
from dslmodel.generators import gen_list
from dslmodel.utils.model_tools import run_dsls
from dslmodel.generators.gen_dslmodel_class import DSLModelClassTemplateSpecificationModel, FieldTemplateSpecificationModel, class_template_str, write_pydantic_class_to_file
from dslmodel.template import render
from dslmodel.utils.str_tools import pythonic_str

In [3]:
# Initialize the necessary environments
init_instant()

# Set the model prompt
model_prompt = ("I need a verbose contact model named ContactModel from the friend of a friend ontology with 20 fields")

# Step 1: Generate field list from the prompt
fields = gen_list(f"{model_prompt}\nOnly list the field names.")
print(fields)

In [5]:
# %%python
# Step 2: Generate field descriptions using run_dsls
tasks = [
    (FieldTemplateSpecificationModel, f"Generate a field named {field} with a useful description")
    for field in fields
]
results = run_dsls(tasks, 10)
print(results)

In [6]:
# Step 3: Instantiate DSLModelClassTemplateSpecificationModel from the prompt
model_inst = DSLModelClassTemplateSpecificationModel.from_prompt(model_prompt, True)

# Step 4: Prepare the template data
template_data = {**model_inst.model_dump(), "fields": results}

# Step 5: Render the Pydantic class from the specification template
rendered_class_str = render(class_template_str, model=template_data)
print(rendered_class_str)

In [ ]:
# Step 6: Write the rendered class to a Python file
filename = f"{pythonic_str(model_inst.class_name)}.py"
write_pydantic_class_to_file(rendered_class_str, filename)
print(filename)

In [12]:
from pathlib import Path
togaf_context = Path("togaf-context.md").read_text()
print(togaf_context)

In [13]:
classes = gen_list(f"{togaf_context}\n\nCreate a list of all classes required")
print(classes)

In [15]:
import dspy
togaf_summary = dspy.Predict("togaf -> togaf_summary")(togaf=togaf_context).togaf_summary
print(togaf_summary)

In [16]:
curr_class = classes[0]
fields = gen_list(f"{togaf_summary}\nOnly list the field names for the {curr_class} of the TOGAF Python Framework")
print(fields)

In [17]:
# Step 2: Generate field descriptions using run_dsls
tasks = [
    (FieldTemplateSpecificationModel, f"Generate a field named {field} with a useful description")
    for field in fields
]
results = run_dsls(tasks, 10)
print(results)

In [20]:
# Step 3: Instantiate DSLModelClassTemplateSpecificationModel from the prompt
model_inst = DSLModelClassTemplateSpecificationModel.from_prompt(togaf_summary, True)

# Step 4: Prepare the template data
template_data = {**model_inst.model_dump(), "fields": results}

# Step 5: Render the Pydantic class from the specification template
rendered_class_str = render(class_template_str, model=template_data)
print(rendered_class_str)

In [21]:
# Step 6: Write the rendered class to a Python file
filename = f"togaf/{pythonic_str(model_inst.class_name)}.py"
write_pydantic_class_to_file(rendered_class_str, filename)
print(filename)